In [160]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import datetime

import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred

In [161]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [164]:
sql="""SELECT s.ticker, max(d.symbol_id), max(d.date_price)
FROM public.symbol s inner join daily_data d on s.id= d.symbol_id
group by s.ticker"""
df_ticker_last_day=pd.read_sql(sql,con=conn_indicator)

In [165]:
df_ticker_last_day

,ticker,max,max
0,BOM500096,9469,2020-03-31
1,BOM500087,9468,2020-03-31
2,BOM500257,9481,2020-03-31
3,BOM500180,9474,2020-03-31
4,BOM500228,9478,2020-03-31
5,BOM500295,9483,2020-03-31
6,BOM500209,9477,2020-03-31
7,BOM500112,9471,2020-03-31
8,BOM500010,9466,2020-03-31
9,BOM500104,9470,2020-03-31


In [166]:
sql="""select date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data where stock_id=71"""
df_stock=pd.read_sql(sql,con=conn_secmaster)

In [167]:
import datetime

In [168]:
sql="select * from indicator"
ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
ind_list

['candle_1', 'candle_2', 'candle_3', 'level', 'anomaly']

In [169]:
symbol_id='BOM500034'
start_date=datetime.datetime(2020,3,31).strftime("%Y-%m-%d")

In [170]:
sql="select s.ticker, s.name, d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data d join symbol s on d.stock_id = s.id where d.date_price = '%s'" %(start_date)
df_price=pd.read_sql(sql,con=conn_secmaster)

In [171]:
sql="select s.ticker, s.name, d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where i.name = '%s' and d.date_price = '%s'" %(indicator_name, start_date)
df_indicator=pd.read_sql(sql,con=conn_indicator)

In [172]:
df_all_ind=pd.DataFrame()
for ind in ind_list:
    print(ind)
    sql="select s.ticker, d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where i.name = '%s' and d.date_price >= '%s'" %(ind, start_date)
    df_indicator=pd.read_sql(sql,con=conn_indicator)
    df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
    interested_columns=list(set(df_indicator.columns)-set(['date','ticker','name']))
    new_column=[ind+"_"+col for col in interested_columns]
    df_indicator.rename(columns=dict(zip(interested_columns,new_column)),inplace=True)
    if df_all_ind.empty:
        df_all_ind=pd.merge(left=df_price, right=df_indicator,left_on=['date','ticker'],right_on=['date','ticker'])
    else:
        df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,left_on=['date','ticker'],right_on=['date','ticker'])

candle_1
candle_2
candle_3
level
anomaly


In [173]:
df_all_ind.columns

Index(['ticker', 'name', 'date', 'open', 'high', 'low', 'close', 'volume',
       'candle_1_level', 'candle_1_pattern_name', 'candle_2_level',
       'candle_2_pattern_name', 'candle_3_level', 'candle_3_pattern_name',
       'level_level', 'level_nearest_resistance_atr_range',
       'level_nearest_resistance_duration', 'level_nearest_resistance_range',
       'level_nearest_support_atr_range', 'level_nearest_support_duration',
       'level_nearest_support_range', 'level_resistance', 'level_support',
       'anomaly_bear', 'anomaly_body_size', 'anomaly_body_size_dev',
       'anomaly_body_size_ema', 'anomaly_bull', 'anomaly_gap',
       'anomaly_gap_dev', 'anomaly_gap_ema', 'anomaly_gap_signal',
       'anomaly_vol_anomaly', 'anomaly_vol_anomaly_2', 'anomaly_vol_dev',
       'anomaly_vol_ema', 'anomaly_wick_bear', 'anomaly_wick_bear_dev',
       'anomaly_wick_bear_ema', 'anomaly_wick_bear_signal',
       'anomaly_wick_bull', 'anomaly_wick_bull_dev', 'anomaly_wick_bull_ema',
       'an

In [174]:
pd.options.display.max_columns = None

In [175]:
df_all_ind[['ticker','name','candle_1_pattern_name','candle_2_pattern_name','anomaly_vol_anomaly','level_nearest_resistance_duration','level_nearest_support_duration']]

,ticker,name,candle_1_pattern_name,candle_2_pattern_name,anomaly_vol_anomaly,level_nearest_resistance_duration,level_nearest_support_duration
0,BOM500010,HOUSING DEVELOPMENT FINANCE CORP.LTD.,DOJI_SPINOFF,BULLISH HARAMI CROSS,0.108561,398,12.0
1,BOM500034,Bajaj Finance Limited,short,,0.524965,504,7.0
2,BOM500087,CIPLA LTD.,,BEARISH (Dark Cloud Cover),0.421986,18,18.0
3,BOM500096,DABUR INDIA LTD.,,,0.039553,11,8.0
4,BOM500104,HINDUSTAN PETROLEUM CORPORATION LTD.,long,,0.263575,13,543.0
5,BOM500112,STATE BANK OF INDIA,,,0.252188,4,7.0
6,BOM500114,Titan Company Limited,,BEARISH ENGULFING,0.015249,439,868.0
7,BOM500124,DR.REDDY&#39;S LABORATORIES LTD.,,,0.212974,26,12.0
8,BOM500180,HDFC Bank Ltd,DOJI_SPINOFF,BULLISH HARAMI CROSS,0.531828,4,7.0
9,BOM500182,HERO MOTOCORP LTD.,DOJI_SPINOFF,BULLISH HARAMI CROSS,0.500605,4,7.0
